# Multitask learning did not perform well in this test, possible because of the poor data overlap. Can transfer learning do better?

In [1]:
import deepchem as dc
import numpy as np
import pandas as pd
import optuna

import cytoxnet.dataprep.io as io
import cytoxnet.dataprep.dataprep as dataprep
import cytoxnet.dataprep.featurize as feat
from cytoxnet.models.models import ToxModel
import cytoxnet.models.opt as opt

In [2]:
fish_df = io.load_data('zhu_rat_LD50')
algea_df = io.load_data('lunghini_algea_EC50')

In [3]:
fish_f = feat.add_features(fish_df, method='ConvMolFeaturizer')
algea_f = feat.add_features(algea_df, method='ConvMolFeaturizer')

In [4]:
fish = dataprep.convert_to_dataset(
    fish_f,
    X_col='ConvMolFeaturizer',
    y_col=[
        'rat_LD50'
    ]
)
algea = dataprep.convert_to_dataset(
    algea_f,
    X_col='ConvMolFeaturizer',
    y_col=[
        'algea_EC50'
    ]
)

In [5]:
fish_normed, fish_transformations = dataprep.data_transformation(
    fish, transformations = ['NormalizationTransformer'],
    to_transform = ['y']
)
algea_normed, algea_transformations = dataprep.data_transformation(
    algea, transformations = ['NormalizationTransformer'],
    to_transform = ['y']
)

In [6]:
algea_dev, algea_test = dataprep.data_splitting(algea_normed, split_type='tt')

Retrieve optimum hyperparameters

In [12]:
model = ToxModel(
    'GraphCNN',
    mode='regression',
    transformers = fish_transformations
)

In [13]:
model.fit(fish_normed, nb_epoch=150)

/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_14:0", shape=(360,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_13:0", shape=(360, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_17:0", shape=(1304,), dtype=int32), values=Tensor("gradient_tape/private

0.033400835990905764

In [14]:
model.transformers = algea_transformations

In [19]:
model.evaluate(algea_test, ['r2_score'])

{'metric-1': 0.300045051853825}

In [33]:
for layer in model._model.model.layers[:-1]:
    layer.trainable = False

In [40]:
model.fit(algea_dev, nb_epoch=10)

0.02308584898710251

In [41]:
model.evaluate(algea_test, ['r2_score'])

{'metric-1': 0.46214152712216694}